In [1]:

import pytorch_lightning
from monai.utils import set_determinism
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    EnsureType,
)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, list_data_collate, decollate_batch, DataLoader
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import random
import nibabel as nib
import numpy as np
import SimpleITK as sitk
from math import floor, ceil

In [3]:
"""Load image and label into numpy array"""
image = nib.load('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/164/164_image.nii.gz')
image_np = np.array(image.dataobj)
#print(image_np.shape)
label = nib.load('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/164/164_label.nii.gz')
label_np = np.array(label.dataobj)
#print(label_np.shape)
#print(np.unique(label_np))
#print(np.unique(image_np))
#print(label_np.dtype)
#plt.imshow(image_np[:,:,48], cmap='gray')
#plt.imshow(label_np[:,:,48], cmap='gray')


#im = nib.load('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/164/164_image.nii.gz')
#im_np = np.array(im.dataobj)
#print(im_np.shape)
#im_sitk = sitk.GetImageFromArray(im_np)
#im_sitk = sitk.ReadImage(data[i][0], imageIO="NiftiImageIO")

ref_img = sitk.Image(96, 512, 512, sitk.sitkFloat32)
im_sitk = sitk.ReadImage('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/164/164_image.nii.gz', imageIO="NiftiImageIO")

Reader = sitk.ImageFileReader(ref=my_img)
Reader.SetFileName('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/164/164_image.nii.gz')
Image = Reader.Execute()
Image.SetDirection((1,0,0,0,0,1,0,1,0))
test = sitk.GetArrayFromImage(Image)
#plt.imshow(test[:,:,48], cmap='gray')
print(Image.GetSize())
print(Image.GetDirection())

resample = sitk.ResampleImageFilter()
resample.SetInterpolator(sitk.sitkNearestNeighbor)
resample.SetOutputDirection(Image.GetDirection())
#print(resample.GetInterpolator())
print(im_sitk.GetSize())
direction = np.array(im_sitk.GetDirection()).reshape(len(Image.GetSize()), -1)
ind = np.argmax(np.abs(direction), axis=0)
new_size = np.array(Image.GetSize())[ind]
resample.SetSize((128, 512, 512))
print(new_size)
#resample.SetSize(new_size.tolist())
#resample.SetReferenceImage(sitk.GetArrayFromImage(image_np))

im_resampled = resample.Execute(im_sitk)
im_re_np = sitk.GetArrayFromImage(im_resampled)
#print(im_re_np.shape)

#plt.imshow(im_np[:,48,:], cmap='gray')
#plt.imshow(im_np[:,:,48], cmap='gray')
#plt.imshow(im_re_np[:,:,48], cmap='gray')

(512, 512, 96)
(512, 512, 96)
[0 1]
[   0    1    2 ... 2037 2049 2084]
int16
(512, 512, 96)
(512, 512, 96)
(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0)
(512, 512, 96)
[512  96 512]
(512, 512, 128)


In [5]:
"""
Simple ITK
"""

images = []
labels = []
for root, dirs, files in os.walk('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/'):
    for file in files:
        if file.endswith("_image.nii.gz"):
            images.append(os.path.join(root, file))
        if file.endswith("_label.nii.gz"):
            labels.append(os.path.join(root, file))
images = sorted(images)
labels = sorted(labels)

data = list(zip(images, labels))

resample = sitk.ResampleImageFilter()
resample.SetSize((96, 512, 512))

resample = sitk.ResampleImageFilter()
resample.SetSize((128, 512, 512))

for i in range(len(data)):
    print(i)
    im = nib.load(data[i][0])
    im_np = np.array(im.dataobj)
    print(im_np.shape)
    #im_sitk = sitk.GetImageFromArray(im_np)
    im_sitk = sitk.ReadImage(data[i][0], imageIO="NiftiImageIO")
    print(im_sitk.GetSize())
    direction = np.array(im_sitk.GetDirection()).reshape(len(Image.GetSize()), -1)
    print(direction)
    im_resampled = resample.Execute(im_sitk)
    im_re_np = sitk.GetArrayFromImage(im_resampled)
    print(im_re_np.shape)
    la = nib.load(data[i][1])
    la_np = np.array(la.dataobj)
    print(la_np.shape)

0
(512, 512, 96)
(512, 512, 96)
[[ 1.          0.          0.        ]
 [ 0.          0.06958133  0.99757628]
 [ 0.         -0.99757628  0.06958132]]
(512, 512, 128)
(512, 512, 96)
1
(290, 320, 64)
(290, 320, 64)
[[ 1.          0.          0.        ]
 [ 0.         -0.04884947  0.99880615]
 [ 0.         -0.99880615 -0.04884946]]
(512, 512, 128)
(290, 320, 64)
2
(512, 512, 84)
(512, 512, 84)
[[ 0.99924386  0.         -0.03888079]
 [ 0.0388808   0.          0.99924386]
 [ 0.         -1.          0.        ]]
(512, 512, 128)
(512, 512, 84)
3
(512, 512, 96)
(512, 512, 96)
[[ 1.          0.          0.        ]
 [ 0.          0.16184001  0.98681701]
 [ 0.         -0.98681701  0.16184   ]]
(512, 512, 128)
(512, 512, 96)
4
(290, 320, 72)
(290, 320, 72)
[[ 1.         0.         0.       ]
 [ 0.         0.0854169  0.9963453]
 [ 0.        -0.9963453  0.0854169]]
(512, 512, 128)
(290, 320, 72)
5
(512, 512, 96)
(512, 512, 96)
[[ 0.99829609  0.         -0.05835171]
 [ 0.05835171  0.          0.9982

In [89]:
"""
Padding
"""

images = []
labels = []
for root, dirs, files in os.walk('/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/'):
    for file in files:
        if file.endswith("_image.nii.gz"):
            images.append(os.path.join(root, file))
        if file.endswith("_label.nii.gz"):
            labels.append(os.path.join(root, file))
images = sorted(images)
labels = sorted(labels)

data = list(zip(images, labels))

for i in range(len(data)):
    print(i)
    print(data[i][0])
    im = nib.load(data[i][0])
    im_np = np.array(im.dataobj)
    print(im_np.shape)
    size = im_np.shape
    im_re_np = np.pad(im_np,
                        ((floor((512 - size[0])/2), ceil((512 - size[0])/2)),
                        (floor((512 - size[1])/2), ceil((512 - size[1])/2)),
                        (floor((104 - size[2])/2), ceil((104 - size[2])/2))),
                        'constant', constant_values=(0, 0))
    print(im_re_np.shape)
    la = nib.load(data[i][1])
    la_np = np.array(la.dataobj)
    print(la_np.shape)
    lsize = la_np.shape
    la_re_np = np.pad(la_np,
                        ((floor((512 - lsize[0])/2), ceil((512 - lsize[0])/2)),
                        (floor((512 - lsize[1])/2), ceil((512 - lsize[1])/2)),
                        (floor((104 - lsize[2])/2), ceil((104 - lsize[2])/2))),
                        'constant', constant_values=(0, 0))
    print(la_re_np.shape)

0
/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/1/1_image.nii.gz
(512, 512, 96)
(512, 512, 104)
(512, 512, 96)
(512, 512, 104)
1
/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/12/12_image.nii.gz
(290, 320, 64)
(512, 512, 104)
(290, 320, 64)
(512, 512, 104)
2
/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/13/13_image.nii.gz
(512, 512, 84)
(512, 512, 104)
(512, 512, 84)
(512, 512, 104)
3
/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/149/149_image.nii.gz
(512, 512, 96)
(512, 512, 104)
(512, 512, 96)
(512, 512, 104)
4
/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/150/150_image.nii.gz
(290, 320, 72)
(512, 512, 104)
(290, 320, 72)
(512, 512, 104)
5
/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/152/152_image.nii.gz
(512, 512, 96)
(512, 512, 104)
(512, 512, 96)
(512, 512, 104)
6
/media/sasank/LinuxStorage/Dropbox (UFL)/FistulaData/Segmentations/153/153_image.nii.gz
(512, 

In [ ]:

img1 = sitk.REadImage(img1.nii.gz)
img2 = sitk.ReadImage(img2.nii.gz)

ref = sitk.Image(96, 512, 512, siktFloat)
ref.SetOrigin(img2.GetOrigin())
ref.SetDirection(img2.GetDirection())
spacing = img2.GetSpacing()
new_spacing = [img2.spacing[0]*img2.size[0]/img1.size[0],
                img2.spacing[1]*img2.size[1]/img1.size[1],
                img2.spacing[2]*img2.size[2]/img1.size[2]]
ref.SetSpacing(new_spacing)

res = sitk.ResampleImageFilter()
res.SetReferenceImage(ref)
img1_resampled = res.Execute(img1)